In [3]:
### imports
import numpy as np
#import tensorflow.keras as keras
import keras
from matplotlib import pyplot as plt
import os
import copy


Using TensorFlow backend.


layers:  concatentate
         bidirectional rnn
         merge with different sizes
         conv2d
         pool2d 
         conv3d
         pool3d
         global pool 2d
         global pool 3d
         seperable conv
         conv transpose
         depthwise conv
         crop1d
         crop2d
         crop3d
         pad1d
         pad2d
         pad3d
         upsampling1d
         upsampling2d
         upsampling3d
         locally connected 1d
         locally connected 2d
         time distributed
 
 static or const?
 broadcasting sizes for merge layers
 find the common dimension and add them that way?


# layer tests

In [3]:
### average pooling 1d test

def arravg(array,numels,offset):
    avg=0
    ct = 0
    for i in range(numels):
        if array[i*offset] > -np.inf:
            avg += array[i*offset]
            ct += 1
    avg = avg/ct
    return avg

def avgpool1d(inputs,pool_size,stride,out_height,in_width):
    k=0
    outputs = np.zeros(out_height*in_width)
    inputs = inputs.flatten(order='C')
    for i in range(out_height):
        inrowidx = k*in_width;
        outrowidx = i*in_width;
        for j in range(in_width):
            outputs[outrowidx+j] = arravg(inputs[inrowidx+j:],pool_size,in_width)
        k += stride
    return outputs

inshape = (10,10)
pool_size=3
stride=1
pad = 'same'
a = keras.layers.Input(inshape)
b = keras.layers.AveragePooling1D(pool_size=pool_size, strides=stride, padding=pad)(a)
model = keras.models.Model(inputs=[a], outputs=[b])

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)

(in_height,in_width) = model.layers[1].input_shape[1:]
(out_height, out_width) = model.layers[1].output_shape[1:]

if pad is 'same':
    pad_along_height = max((out_height - 1) * stride +
                        pool_size - in_height, 0)
    pad_top = int(pad_along_height // 2)
    pad_bottom = int(pad_along_height - pad_top)
elif pad is 'valid':
    pad_top=0
    pad_bottom=0
padded_in_height = in_height + pad_top + pad_bottom


y1 = np.concatenate((-np.inf*np.ones((pad_top,in_width)),x,-np.inf*np.ones((pad_bottom,in_width))),0)
y2 = avgpool1d(y1,pool_size,stride,out_height,in_width)
np.mean(np.abs(y.flatten(order='C')-y2.flatten(order='C')))


2.0694171458668366e-08

In [4]:
### max pooling 1d test
def arrmax(array,numels,offset):
    maxval=array[0];
    for i in range(numels):
        if array[i*offset]>maxval:
            maxval = array[i*offset]
    return maxval  

def maxpool1d(inputs,pool_size,stride,out_height,in_width):
    k=0
    outputs = np.zeros(out_height*in_width)
    inputs = inputs.flatten(order='C')
    for i in range(out_height):
        inrowidx = k*in_width;
        outrowidx = i*in_width;
        for j in range(in_width):
            outputs[outrowidx+j] = arrmax(inputs[inrowidx+j:],pool_size,in_width)
        k += stride
    return outputs

inshape = (8,23)
pool_size=2
stride=1
pad = 'same'
a = keras.layers.Input(inshape)
b = keras.layers.MaxPooling1D(pool_size=pool_size, strides=stride, padding=pad)(a)
model = keras.models.Model(inputs=[a], outputs=[b])

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)

(in_height,in_width) = model.layers[1].input_shape[1:]
(out_height, out_width) = model.layers[1].output_shape[1:]

if pad is 'same':
    pad_along_height = max((out_height - 1) * stride +
                        pool_size - in_height, 0)
    pad_top = int(pad_along_height // 2)
    pad_bottom = int(pad_along_height - pad_top)
elif pad is 'valid':
    pad_top=0
    pad_bottom=0
padded_in_height = in_height + pad_top + pad_bottom

y1 = np.concatenate((-np.inf*np.ones((pad_top,in_width)),x,-np.inf*np.ones((pad_bottom,in_width))),0)
y2 = maxpool1d(y1,pool_size,stride,out_height,in_width)
np.mean(np.abs(y.flatten(order='C')-y2.flatten(order='C')))

1.1858545135591115e-08

In [5]:
### conv1d test

def conv1d(x,kernel,bias,out_height,out_width,kernel_size,in_width,padded_in_height,stride,dilation):
    x = x.flatten(order='C')
    kernel = kernel.flatten(order='C')
    out_size = out_height*out_width
    output = np.zeros(out_size)
   
    for p in range(out_height):
        outrowidx = p*out_width
        for k in range(out_width):
            for z in range(kernel_size):
                kernelidx = z*in_width*out_width
                for q in range(in_width):
                    inheightidx = q*out_width
                    output[outrowidx+k] += kernel[kernelidx+ inheightidx+ k]*x[(p*stride+ z*dilation)*in_width+q];
            output[outrowidx+k] += bias[k]
    
    return output

inshape = (10,15)
pool_size=3
stride=1
dilation = 1
kernel_size = 4
num_filters = 9
pad = 'same'
a = keras.layers.Input(inshape)
b = keras.layers.Conv1D(filters=num_filters, dilation_rate=dilation, kernel_size=kernel_size, strides=stride, padding=pad)(a)
model = keras.models.Model(inputs=[a], outputs=[b])

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)

(in_height,in_width) = model.layers[1].input_shape[1:]
(out_height, out_width) = model.layers[1].output_shape[1:]

if pad is 'causal':
    pad_along_height = dilation*(kernel_size-1)
    pad_top = pad_along_height
    pad_bottom = 0
elif pad is 'same':
    pad_along_height = max((out_height - 1) * stride +
                        kernel_size - in_height, 0)
    pad_top = int(pad_along_height // 2)
    pad_bottom = int(pad_along_height - pad_top)
elif pad is 'valid':
    pad_top=0
    pad_bottom=0
padded_in_height = in_height + pad_top + pad_bottom

kernel = model.layers[1].get_weights()[0]
bias = model.layers[1].get_weights()[1]
y1 = np.concatenate((np.zeros((pad_top,in_width)),x,np.zeros((pad_bottom,in_width))),0)
y2 = conv1d(y1,kernel,bias,out_height,out_width,kernel_size,in_width,padded_in_height,stride,dilation)
print(np.mean(np.abs(y.flatten(order='C')-y2.flatten(order='C'))))
print(model.layers[1].get_weights()[0].shape)
print(model.layers[1].get_weights()[1].shape)

Instructions for updating:
Colocations handled automatically by placer.
6.154065818762976e-08
(4, 15, 9)
(9,)


In [6]:
### simple RNN test
def rnn(inputs,weights,recurrent_weights,bias):
    state = np.zeros(bias.shape[0])
    for i in range(inputs.shape[0]):
        state = rnncell(inputs[i,:],state,weights,recurrent_weights,bias)
    return state
   
def rnncell(inputs,state,weights,recurrent_weights,bias):
    prev = state
    h = inputs@weights+bias
    output = h + prev@recurrent_weights
    output = np.tanh(output)
    
    return output

insize = (10,33)
a = keras.layers.Input(insize)
b = keras.layers.SimpleRNN(12, dropout=.5, recurrent_dropout=.5)(a)

model = keras.models.Model(inputs=[a], outputs=[b])
x = np.random.random(insize)
x1 = x[np.newaxis,...]
y = model.predict(x1)

model.layers[1].output_shape

weights = model.layers[1].get_weights()[0]
recurrent_weights = model.layers[1].get_weights()[1]
bias = model.layers[1].get_weights()[2]
a = rnn(x,weights,recurrent_weights,bias)
np.max(np.abs(a-y))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2.3231411758195541e-07

In [8]:
### GRU test

def gru(inputs, kernel, recurrent_kernel, input_bias, recurrent_bias, reset_after, units, go_backwards, return_sequences):
    inp = inputs.flatten()
    state = np.zeros(units)
    output = np.zeros((inputs.shape[0]*units))
    if go_backwards:
        for i in range(inputs.shape[0]-1,-1,-1):
            state = grucell(inp[i*inputs.shape[1]:(i+1)*inputs.shape[1]], state, kernel, recurrent_kernel, input_bias, recurrent_bias, reset_after, units)
            if return_sequences:
                for j in range(units):
                    output[(inputs.shape[0]-1-i)*units+j] = state.flatten()[j]
                
    else:
        for i in range(inputs.shape[0]):
            state = grucell(inp[i*inputs.shape[1]:(i+1)*inputs.shape[1]], state, kernel, recurrent_kernel, input_bias, recurrent_bias, reset_after, units)
            if return_sequences:
                for j in range(units):
                    output[i*units+j] = state.flatten()[j]
    if return_sequences:
        return output
    else:
        return output
    

def grucell(inputs, state, kernel, recurrent_kernel, input_bias, recurrent_bias, reset_after, units):
    kernel_z = kernel[:, :units]
    recurrent_kernel_z = recurrent_kernel[:, :units]
    # reset gate
    kernel_r = kernel[:, units: units * 2]
    recurrent_kernel_r = recurrent_kernel[:,units:units * 2]
    # new gate
    kernel_h = kernel[:, units * 2:]
    recurrent_kernel_h = recurrent_kernel[:, units * 2:]
    
    input_bias_z = input_bias[:units]
    input_bias_r = input_bias[units: units * 2]
    input_bias_h = input_bias[units * 2:]
        
    recurrent_bias_z = recurrent_bias[:units]
    recurrent_bias_r = recurrent_bias[units: units * 2]
    recurrent_bias_h = recurrent_bias[units * 2:]
    
    
    x_z = inputs@kernel_z + input_bias_z
    x_r = inputs@kernel_r + input_bias_r
    x_h = inputs@kernel_h + input_bias_h
    
    h_tm1 = state

            
    recurrent_z = h_tm1@recurrent_kernel_z
    recurrent_r = h_tm1@recurrent_kernel_r
    recurrent_z = recurrent_z + recurrent_bias_z
    recurrent_r = recurrent_r + recurrent_bias_r

    recurrent_z = np.tanh(x_z + recurrent_z)
    recurrent_r = np.tanh(x_r + recurrent_r)

    # reset gate applied after/before matrix multiplication
    if reset_after:
        recurrent_h = h_tm1@recurrent_kernel_h + recurrent_bias_h
        recurrent_h = recurrent_r * recurrent_h
    else:
        recurrent_h = (recurrent_r * h_tm1)@recurrent_kernel_h
    hh = np.tanh(x_h + recurrent_h)
    h = recurrent_z * h_tm1 + (1 - recurrent_z) * hh
    return h

reset_after = True
go_backwards = True
return_sequences = True
units=12
insize = (10,33)
a = keras.layers.Input(insize)
b = keras.layers.GRU(units, reset_after=reset_after, activation='tanh', recurrent_activation='tanh',\
                    bias_initializer='zeros', go_backwards=go_backwards, return_sequences=return_sequences)(a)

model = keras.models.Model(inputs=[a], outputs=[b])
x = np.random.random(insize)
x1 = x[np.newaxis,...]
y = model.predict(x1)

kernel = model.layers[1].get_weights()[0]
recurrent_kernel = model.layers[1].get_weights()[1]
bias = model.layers[1].get_weights()[2]
if reset_after:
    input_bias = bias[0,:]
    recurrent_bias = bias[1,:]
else:
    recurrent_bias = np.zeros(3*units)
    input_bias = bias

y2 = gru(x, kernel, recurrent_kernel, input_bias, recurrent_bias, reset_after, units, go_backwards, return_sequences)
np.max(np.abs(y2.flatten()-y.flatten())) 



3.2177304026825837e-07

In [9]:
### dropout test
inshape = (10,20)
a = keras.layers.Input(inshape)
b = keras.layers.Dense(40, use_bias=False)(a)
c = keras.layers.Dropout(.9)(b)
d = keras.layers.Dense(40, use_bias=False)(c)
model = keras.models.Model(inputs=a, outputs=d)

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)

y2 = (x@model.layers[1].get_weights()[0])@model.layers[3].get_weights()[0]

np.max(np.abs(y2-y)) 



4.0230503772775705e-07

In [10]:
### flatten / reshape test
x = np.array(range(12)).reshape((3,4))
x1 = np.reshape(x, (1, -1))
np.max(np.abs(x1.flatten() - x.flatten()))

0

In [11]:
### reshape test
inshape = (20,16)
a = keras.layers.Input(inshape)
b = keras.layers.Reshape((8,2,2,10))(a)

model = keras.models.Model(inputs=a, outputs=b)

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)


np.max(np.abs(x.flatten()-y.flatten()))


2.94372511921992e-08

In [12]:
### transpose testing
def transp(a,ndim,olddim,permute):
    a = a.flatten()
    b = np.zeros(a.shape)
    oldidx2d = np.array([0,0])
    oldidx3d = np.array([0,0,0])
    newidx2d = np.array([0,0])
    newidx3d = np.array([0,0,0])

#    b[i,j,k] = a[i * (dim2 * dim3) + (j * dim3) + k]
#    b[i,j] = a[i*dim2 + j]
    if ndim==1:
        return a # no need to transpose a 1d array
    if ndim==2:
        oldrows = olddim[0]
        oldcols = olddim[1]
        newrows = olddim[permute[0]]
        newcols = olddim[permute[1]]
        for i in range(oldrows):
            for j in range(oldcols):
                oldidx2d = [i,j]
                newidx2d = [oldidx2d[permute[0]],oldidx2d[permute[1]]]
                b[newidx2d[0]*newcols + newidx2d[1]] = a[oldidx2d[0]*oldcols + oldidx2d[1]]
        return b
    if ndim==3:
        oldrows = olddim[0]
        oldcols = olddim[1]
        oldchan = olddim[2]
        newrows = olddim[permute[0]]
        newcols = olddim[permute[1]]
        newchan = olddim[permute[2]]
        for i in range(oldrows):
            for j in range(oldcols):
                for k in range(oldchan):
                    oldidx3d = [i,j,k]
                    newidx3d = [oldidx3d[permute[0]],oldidx3d[permute[1]],oldidx3d[permute[2]]]
                    b[newidx3d[0]*newcols*newchan + newidx3d[1]*newchan + newidx3d[2]] =\
                        a[oldidx3d[0]*oldcols*oldchan + oldidx3d[1]*oldchan + oldidx3d[2]]
        return b
    if ndim==4:
        oldrows = olddim[0]
        oldcols = olddim[1]
        oldchan = olddim[2]
        oldtime = olddim[3]
        newrows = olddim[permute[0]]
        newcols = olddim[permute[1]]
        newchan = olddim[permute[2]]
        newtime = olddim[permute[3]]
        for i in range(oldrows):
            for j in range(oldcols):
                for k in range(oldchan):
                    for l in range(oldtime):
                        oldidx4d = [i,j,k,l]
                        newidx4d = [oldidx4d[permute[0]],oldidx4d[permute[1]],oldidx4d[permute[2]],oldidx4d[permute[3]]]
                        b[newidx4d[0]*newcols*newchan*newtime + newidx4d[1]*newchan*newtime + newidx4d[2]*newtime + newidx4d[3]] =\
                            a[oldidx4d[0]*oldcols*oldchan*oldtime + oldidx4d[1]*oldchan*oldtime + oldidx4d[2]*oldtime + oldidx4d[3]]
        return b

inshape = (5,7,8,12)
permute = np.array((3,2,1,4))
ndim = 4
l1 = keras.layers.Input(inshape)
l2 = keras.layers.Permute(permute)(l1)
model = keras.models.Model(inputs=l1, outputs=l2)

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)

xt = transp(x,ndim,inshape,permute-1)

np.max(np.abs(xt-y.flatten()))
    

2.9796300538009746e-08

In [13]:
inshape = (4,5)
a = keras.layers.Input(inshape)
b = keras.layers.Dense(10)(a)
model = keras.models.Model(inputs=a, outputs=b)
model.layers[1].get_weights()[0].shape

(5, 10)

# misc testing and notes

In [14]:
### indexing testing
dim0=9
dim1=48
dim2=19
b = np.random.random((dim0,dim1,dim2))
a = b.flatten(order='C')
idx = (7,21,5)
b[idx] - a[idx[0] * (dim1 * dim2) + (idx[1] * dim2) + idx[2]]

0.0

In [15]:
### input/output sizing with filters/padding
#out_height = ceil(float(in_height) / float(strides[1]))
#out_width  = ceil(float(in_width) / float(strides[2]))

#pad_along_height = max((out_height - 1) * strides[1] +
#                    filter_height - in_height, 0)
#pad_along_width = max((out_width - 1) * strides[2] +
#                   filter_width - in_width, 0)
#pad_top = pad_along_height // 2
#pad_bottom = pad_along_height - pad_top
#pad_left = pad_along_width // 2
#pad_right = pad_along_width - pad_left




In [16]:
### pydot graph testing
#graph = pydot.graph_from_dot_data(str())
graph = keras.utils.vis_utils.model_to_dot(model)
nodes = graph.get_nodes()
edges = graph.get_edges()
# for edge in edges:
#     print(edge)
# for node in nodes:
#     print(node)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [17]:
### kgraph stuff

class keras_node():
    def __init__(self, pydot_node):
        self.pydot_node = pydot_node
        self.ID = pydot_node.to_string().split()[0]
        self.type = pydot_node.to_string().split()[2][:-3]
        self.name = pydot_node.to_string().split()[1][8:-1]
        self.inputs = []
        self.outputs = []
    
    def find_layer_idx(self,model):
        """finds the index into model.layers corresponding to the node"""
        for i, layer in enumerate(model.layers):
            if layer.name == self.name:
                self.layer_idx = i
                return
        self.layer_idx = None
            
class keras_edge():
    def __init__(self, pydot_edge):
        self.pydot_edge = pydot_edge
        self.start_id = pydot_edge.to_string().split()[0]
        self.end_id = pydot_edge.to_string().split()[2][:-1]

class keras_graph():
    def __init__(self, model):
        self.model = model
        self.pydot_graph = keras.utils.vis_utils.model_to_dot(model)
        self.edges = self.parse_edges(self.pydot_graph)
        self.nodes = self.parse_nodes(self.pydot_graph)
        self.input_nodes = []
        self.output_nodes = []
        
    def parse_edges(self,pydot_graph):
        """converts pydot edges to keras_graph edges"""
        edges = []
        for edge in pydot_graph.get_edges():
            edges += [keras_edge(edge)]
        return edges
    
    def parse_nodes(self,pydot_graph):
        """converts pydot nodes to keras_graph nodes"""
        nodes = []
        for i, node in enumerate(pydot_graph.get_nodes()[1:]):
            nodes += [keras_node(node)]
            nodes[i].find_layer_idx(self.model)
        return nodes
    
    def parse_connections(self):
        for edge in self.edges:
            self.find_node_from_id(edge.start_id).outputs += [self.find_node_from_id(edge.end_id)]
            self.find_node_from_id(edge.end_id).inputs += [self.find_node_from_id(edge.start_id)]

    def find_node_from_id(self,ID):
        """finds the node associated with a given ID"""
        for node in self.nodes:
            if node.ID == ID:
                return node
        return None
    
    def find_io_nodes(self):
        for node in self.nodes:
            if len(node.inputs) == 0:
                self.input_nodes += [node]
            if len(node.outputs) == 0:
                self.output_nodes += [node]
            
def write_model(model):
    kgraph = keras_graph(model)
    kgraph.parse_connections()
    kgraph.find_io_nodes()

    written_nodes = []
    unwritten_nodes = copy.deepcopy(kgraph.nodes)
    while len(unwritten_nodes)>0:
        for node in unwritten_nodes:
            if set(node.inputs).issubset(written_nodes):
                write_layer(model.layers[node.layer_idx],node)
                written_nodes.append(node)
                unwritten_nodes.remove(node)
        
def write_layer(layer,node):
    print(layer.name)
    

In [18]:
### check and io functions

def get_all_io_names(model):
    def flatten(x):
        if isinstance(x, list) or isinstance(x, tuple):
            return [a for i in x for a in flatten(i)]
        else:
            return [x]
    
    a = [get_layer_io_names(layer) for layer in model.layers]
    return list(set(flatten(a)))

def get_layer_io_names(layer):
    inputs = []
    num_inputs = 0
    error = False
    while not error:
        try:
            layer.get_input_at(num_inputs)
            num_inputs +=1
        except:
            error = True
    
    outputs = []
    num_outputs = 0
    error = False
    while not error:
        try:
            layer.get_output_at(num_outputs)
            num_outputs +=1
        except:
            error = True
    # num_inputs>1 -> shared layer
    for i in range(num_inputs):
        # is the input a list?
        if isinstance(layer.get_input_at(i), list):
            temp_list = []
            list_length = len(layer.get_input_at(i))
            for j in range(list_length):
                name = str(layer.get_input_at(i)[j]).split()[0].split('"')[1].split('/')[0].split(':')[0]
                temp_list.append(name)
            inputs.insert(i,temp_list)
        else:
            name = str(layer.get_input_at(i)).split()[0].split('"')[1].split('/')[0].split(':')[0]
            inputs.insert(i,name)
            
    for i in range(num_outputs):
        # is the output a list?
        if isinstance(layer.get_output_at(i), list):
            temp_list = []
            list_length = len(layer.get_output_at(i))
            for j in range(list_length):
                name = str(layer.get_output_at(i)[j]).split()[0].split('"')[1].split('/')[0].split(':')[0]
                temp_list.append(name)
            outputs.insert(i,temp_list)
        else:
            name = str(layer.get_output_at(i)).split()[0].split('"')[1].split('/')[0].split(':')[0]
            outputs.insert(i,name)

    return (inputs, outputs)

def is_valid_c_name(name):
    allowed_chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_123456789'
    allowed_starting_chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_'
    if not set(name).issubset(allowed_chars):
        return False
    if not set(name[0]).issubset(allowed_starting_chars):
        return False
    return True

def name_check(model):
    for layer in model.layers:
        assert (is_valid_c_name(layer.name)), "layer name '" + layer.name + "' is not a valid C name"

def layer_type(layer):
    return str(layer.__class__).split('.')[-1][0:-2]
def layers_supported_check(model):
    supported_layers = ['Dense','LSTM','Conv1D','InputLayer','MaxPooling1D','AveragePooling1D',\
                        'GlobalMaxPooling1D','GlobalAveragePooling1D','Add','Multiply','Average',\
                        'Maximum','Minimum','LeakyReLU','ELU','ThresholdedReLU','ReLU']
    for layer in model.layers:
        assert(layer_type(layer) in supported_layers), "layer type '" + \
        layer_type(layer) + "' is not supported at this time"
        
def activation_supported_check(model):
    supported_activations = ['linear', 'relu','softmax','softplus','softsign','relu','tanh',\
                             'sigmoid','hard_sigmoid','exponential' ]
    
    for layer in model.layers:
        if 'activation' in layer.get_config():
            assert(layer.get_config()['activation'] in supported_activations), \
            "activation type '" + layer.get_config()['activation'] + "' is not supported at this time"
        if 'recurrent_activation' in layer.get_config():
            assert(layer.get_config()['recurrent_activation'] in supported_activations), \
            "recurrent activation type '" + layer.get_config()['recurrent_activation'] + "' is not supported at this time"


In [19]:
### max pooling 2d

def max_pool2d(inputs,pool_size,stride,outshp):
    channels = inputs.shape[2]
    fin = inputs.flatten()
    outsize = np.prod(outshp)
    outputs = np.zeros(outsize)
    for i in range(channels):
        k=0
        for j in range(0,outshp[1]*channels,channels):
            m=0
            for l in range(0, outsize, channels*outshp[1]):
                outputs[l+j+i] = fin[m+k+i]
                for n in range(0,pool_size[1]*channels, channels):
                    for p in range(0,pool_size[0]*channels*inputs.shape[1],channels*inputs.shape[1]):
                        if (outputs[l+j+i] < fin[m+k+i+n+p]):
                            outputs[l+j+i] = fin[m+k+i+n+p]
                m+=channels*inputs.shape[1]*stride[0]
            k+=channels*stride[1]
    return outputs
inshp = (5,6,7)
pool_size = (3,3)
stride = (3,1)
a = keras.layers.Input(inshp)
b = keras.layers.MaxPooling2D(pool_size=pool_size, strides=stride)(a)
model = keras.models.Model(inputs=a,outputs=b)

x = np.random.random(inshp)
y = model.predict(x[np.newaxis,...])
y = y[0,:]
y1 = max_pool2d(x,pool_size,stride,y.shape)
np.max(np.abs(y.flatten()-y1.flatten()))

2.7442391581367076e-08

In [39]:
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(8, (3, 3), padding='same',
                                  input_shape=(32, 32, 3)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Conv2D(8, (3, 3)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Conv2D(8, (3, 3), padding='same'))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Conv2D(8, (3, 3)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(20))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(10))
    model.add(keras.layers.Activation('softmax'))

In [42]:
model.layers[-].output_shape

(None, 20)

In [31]:
model_inputs, model_outputs = get_model_io_names(model)
written_io = set(model_inputs)
unwritten_io = set(get_all_io_names(model)) - written_io
count = 0
while len(unwritten_io) > 0:
    count += 1
    if count>50:
        break
    for layer in model.layers:
        layer_inputs, layer_outputs = get_layer_io_names(layer)
        for i, (inp, outp) in enumerate(zip(layer_inputs, layer_outputs)):
            if (set(flatten(inp)).issubset(written_io) and
                    set(flatten(outp)).issubset(unwritten_io))or \
                    layer_type(layer) == 'InputLayer':
                print('Writing layer ', outp)
                written_io |= set(flatten(inp))
                written_io |= set(flatten(outp))
                unwritten_io -= set(flatten(inp))
                unwritten_io -= set(flatten(outp))
                print('written: ',written_io)
                print('unwritten: ',unwritten_io)

Writing layer  conv2d_1
written:  {'conv2d_1_input', 'conv2d_1'}
unwritten:  {'dropout_3', 'conv2d_4', 'activation_1', 'flatten_1', 'dense_5', 'activation_4', 'activation_5', 'conv2d_3', 'max_pooling2d_3', 'activation_2', 'max_pooling2d_2', 'conv2d_2', 'dense_4', 'activation_6', 'dropout_2', 'dropout_4', 'activation_3'}
Writing layer  activation_1
written:  {'conv2d_1_input', 'conv2d_1', 'activation_1'}
unwritten:  {'dropout_3', 'conv2d_4', 'flatten_1', 'dense_5', 'activation_4', 'activation_5', 'conv2d_3', 'max_pooling2d_3', 'activation_2', 'max_pooling2d_2', 'conv2d_2', 'dense_4', 'activation_6', 'dropout_2', 'dropout_4', 'activation_3'}
Writing layer  conv2d_2
written:  {'conv2d_1_input', 'conv2d_2', 'conv2d_1', 'activation_1'}
unwritten:  {'dropout_3', 'conv2d_4', 'flatten_1', 'dense_5', 'activation_4', 'activation_5', 'conv2d_3', 'max_pooling2d_3', 'activation_2', 'max_pooling2d_2', 'dense_4', 'activation_6', 'dropout_2', 'dropout_4', 'activation_3'}
Writing layer  activation_2
wr

In [32]:
count

1

In [24]:
def k2c_bias_add(A,b):
    af = A.flatten()
    c = np.zeros(af.size)
    for i in range(0,af.size,b.size):
        for j in range(b.size):
            c[i+j] = af[i+j] + b[j]
    return c
A = np.random.random((4,5,6,10))
b = np.arange(10)
c1 = A+b
c2 = k2c_bias_add(A,b)
np.max(np.abs(c1.flatten()-c2.flatten()))
#print(c1)

0.0

In [62]:
def k2c_sub2idx(sub, shape, ndim):
  #/* converts from subscript to linear indices in row major order */
    idx = 0
    temp = 0;
    for i in range(ndim):
        temp = sub[i];
        for j in range(ndim-1,i,-1):
            temp *= shape[j]
        idx += temp;
    return int(idx)

def k2c_idx2sub(idx,shape,ndim):
    sub = np.zeros(ndim)
    a = np.flip(np.cumprod(shape))
    for j in range(ndim-1,-1,-1):
        sub[j] = idx % shape[j]
        idx = idx // shape[j]
    return tuple(sub.astype(int))

def k2c_permute(a,pdims):
    pdims = np.array(pdims).astype(int)
    shp = a.shape
    print(shp)
    newshp = np.array(shp)[pdims]
    print(newshp)
    numels = a.size
    af = a.flatten()
    b = np.zeros(numels)
    for i in range(numels):
        aidx = i
        #asub = np.unravel_index(i,shp)
        #print(asub)
        asub = tuple(k2c_idx2sub(i,shp,a.ndim))
        bsub = tuple(np.array(asub)[pdims])
        #print(bsub)
        bidx = k2c_sub2idx(bsub,newshp,a.ndim)
        b[bidx] = af[i]
    return b


inshape = (10,40,30)
permute = np.array((3,1,2))
ndim = 3
l1 = keras.layers.Input(inshape)
l2 = keras.layers.Permute(permute)(l1)
model = keras.models.Model(inputs=l1, outputs=l2)

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)

xt = k2c_permute(x,permute-1)

np.max(np.abs(xt.flatten()-y.flatten()))
np.max(np.abs(xt.flatten()-x.flatten()))

# sub = (0,1,2,6)
# shp = (9,4,5,7)
# ndim = len(shp)

# idx1 = np.ravel_multi_index(sub,shp)
# idx2 = k2c_sub2idx(sub,shp,ndim)
# sub1 = np.unravel_index(idx1,shp)
# sub2 = k2c_idx2sub(idx1,shp,ndim)
# print(idx1,idx2)
# print(sub1-np.array(sub2))


(10, 40, 30)
[30 10 40]


0.9903031587526856

In [238]:
inshape = (4,3,6,5)
axes = (2,)
flipped=False
a = np.random.random(inshape)
def k2c_reshape(a,axes,flipped):
    shape_a = a.shape
    axes = [i if i >= 0 else i + len(shape_a) for i in axes]
    free = [i for i in range(len(shape_a)) if i not in axes]
    free_dims = [shape_a[i] for i in free]
    prod_free = int(np.prod([shape_a[i] for i in free]))
    prod_axes = int(np.prod([shape_a[i] for i in axes]))
    perm = list(axes) + free if flipped else free + list(axes)
    new_shape = [prod_axes, prod_free] if flipped else [prod_free, prod_axes]
    reshaped_a = np.reshape(np.transpose(a, perm), new_shape)
    return reshaped_a, free_dims, prod_free,prod_axes

def k2c_dot(a,b,a_axes,b_axes,normalize=False):
    a_reshape, a_free_dims, free_axesA, prod_axesA = k2c_reshape(a, a_axes,False)
    b_reshape, b_free_dims, free_axesB, prod_axesB = k2c_reshape(b, b_axes, True)
    print(free_axesA,prod_axesA,free_axesB,prod_axesB)
    
#     if normalize:
#         y = np.max(np.sum(a_reshape ** 2, a_axes, keepdims=True), a_axes, keepdims=True)
#         print(y)
#         print(y.shape)
#         a_reshape = a_reshape / np.sqrt(y)
#         y = np.max(np.sum(b_reshape ** 2, b_axes, keepdims=True), b_axes, keepdims=True)
#         b_reshape = b_reshape / np.sqrt(y)
#         print(y)
#         print(y.shape)
    if (normalize):
        reshapeA = a_reshape.flatten()
        reshapeB = b_reshape.flatten()
        for i in range(free_axesA):
            s = 0
            for j in range(prod_axesA):
                s += reshapeA[i*prod_axesA + j]**2
            inorm = 1.0/np.sqrt(s);
            for j in range(prod_axesA):
                reshapeA[i*prod_axesA+j] *= inorm
        for i in range(free_axesB):
            s = 0
            for j in range(prod_axesB):
                s += reshapeB[i+free_axesB*j]**2
            inorm = 1.0/np.sqrt(s);
            for j in range(prod_axesB):
                reshapeB[i+free_axesB*j] *= inorm 
        a_reshape = reshapeA.reshape(free_axesA,prod_axesA)
        b_reshape = reshapeB.reshape(prod_axesB, free_axesB)
    ab_matmul = a_reshape@b_reshape
    return np.reshape(ab_matmul, a_free_dims + b_free_dims)
   
a_axes = (1,)
b_axes = (0,)
dotaxes = (2,1)
inshape1 = (5,9)
inshape2 = (9,5)
i1 = keras.layers.Input(inshape1)
i2 = keras.layers.Input(inshape2)
d = keras.layers.Dot(axes=dotaxes, normalize=True)([i1,i2])
model = keras.models.Model(inputs=[i1,i2], outputs = d)
a = np.random.random(inshape1)
b = np.random.random(inshape2)
ak = a[np.newaxis,...]
bk = b[np.newaxis,...]
z = model.predict([ak,bk])
x = np.tensordot(a,b,(a_axes,b_axes))
print(x.shape)
C = np.zeros(x.shape)
y = k2c_dot(a,b,a_axes,b_axes,True)
print('np vs k2c',np.max(np.abs(x-y)))
print('k vs np', np.max(np.abs(z-x)))
print('k vs k2c', np.max(np.abs(z-y)))



(5, 5)
5 9 5 9
np vs k2c 2.029779810366704
k vs np 2.029779856225161
k vs k2c 2.3475072030709754e-07


In [242]:
np.array(model.layers[2].get_config()['axes']) -1

array([1, 0])

# # model testing

In [51]:
### model testing

inshape = (13,10)
pool_size=3
stride=1
dilation=1
num_filters=10
kernel_size=3
pad = 'same'
a = keras.layers.Input(inshape)
b = keras.layers.Dense(10)(a)
c = keras.layers.AveragePooling1D(pool_size=pool_size, strides=stride, padding=pad)(b)
d = keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, strides=stride, padding=pad, dilation_rate=dilation)(c)
e = keras.layers.Input((12,10))
f = keras.layers.Dense(15)(e)
g = keras.layers.LSTM(10)(f)
h = keras.layers.add([g,e])
model = keras.models.Model(inputs=[a,e], outputs=[h])

# a = keras.layers.Input(inshape)
# b = keras.layers.Input(inshape)
# c = keras.layers.LSTM(20)
# d = c(a)
# e = c(b)
# model = keras.models.Model(inputs=[a,b], outputs=[d,e])

#print(model.layers[1].input_shape)
#print(model.layers[1].output_shape)
#print(model.layers[1].get_weights()[0].shape)
#model.layers[2].get_config()
model.layers[-1].input_shape


[(None, 10), (None, 12, 10)]

In [291]:
### pad2d
inshp = (3,4,2)
a = keras.layers.Input(inshp)
b = keras.layers.ZeroPadding2D(padding=((1,2),(3,4)))(a)
model = keras.models.Model(inputs=a, outputs=b)
x = np.ones(inshp)
x = x[np.newaxis,...]
y = model.predict(x)
y = np.squeeze(y)
y.shape    

def pad2d(inputs,fill,pad):
    in_height = inputs.shape[0]
    in_width = inputs.shape[1]
    in_channels = inputs.shape[2]
    pad_top = pad[0]
    pad_bottom = pad[1]
    pad_left = pad[2]
    pad_right = pad[3]
    output = np.zeros((in_height+pad_top+pad_bottom)*(in_width+pad_left+pad_right)*(in_channels))
    fin = inputs.flatten()
    offset = in_channels*(pad_left+pad_right+in_width)*pad_top + in_channels*pad_left
    num = in_channels*in_width;
    for i in range(in_height):
        for j in range(num):
            output[offset+j] = fin[i*num+j]
        offset += num+in_channels*(pad_left+pad_right)
    return output
x = np.ones(inshp)
x1= x[np.newaxis,...]
y1 = model.predict(x1)
y1 = np.squeeze(y1)
y = pad2d(x,0,[1,2,3,4])
np.max(np.abs(y1.flatten()-y.flatten()))

0.0

In [308]:
def k2c_sub2idx(sub, shape, ndim):
  #/* converts from subscript to linear indices in row major order */
    idx = 0
    temp = 0;
    for i in range(ndim):
        temp = sub[i];
        for j in range(ndim-1,i,-1):
            temp *= shape[j]
        idx += temp;
    return int(idx)

def conv2d(inputs,kernel,bias,stride,dilation,output):
    out_rows = output.shape[0]
    out_cols = output.shape[1]
    out_channels= output.shape[2]
    in_channels = inputs.shape[2];
    fin = inputs.flatten()
    fkernel = kernel.flatten()
    foutput = np.zeros(output.size)
   
    for x0 in range(out_rows):
        for x1 in range(out_cols):
            for k in range(out_channels):
                for z0 in range(kernel.shape[0]):
                    for z1 in range(kernel.shape[1]):
                        for q in range(in_channels):
                            outsub = [x0,x1,k]
                            inpsub = [x0*stride[0] + dilation[0]*z0, x1*stride[1] + dilation[1]*z1, q]
                            kersub = [z0,z1,q,k]
                            foutput[k2c_sub2idx(outsub,output.shape,output.ndim)] += \
                            fkernel[k2c_sub2idx(kersub,kernel.shape,kernel.ndim)]*\
                            fin[k2c_sub2idx(inpsub,inputs.shape,inputs.ndim)]
    for i in range(0,output.size,bias.size):
        for j in range(bias.size):
            foutput[i+j] += bias[j]
    return foutput
inshape = (10,15,3)
stride=(1,1)
dilation = (2,2)
kernel_size = (2,4)
num_filters = 15
padding = 'same'
a = keras.layers.Input(inshape)
b = keras.layers.Conv2D(filters=num_filters, dilation_rate=dilation, kernel_size=kernel_size, strides=stride, padding=padding)(a)
model = keras.models.Model(inputs=[a], outputs=[b])
layer = model.layers[1]

x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y = model.predict(x1)
y = np.squeeze(y)
if padding is 'same':
    pad_along_height = dilation[0]*(kernel_size[0]-1)
    pad_top = int(pad_along_height // 2)
    pad_bottom = int(pad_along_height - pad_top)
    pad_along_width = dilation[1]*(kernel_size[1]-1)
    pad_left = pad_along_width//2
    pad_right = pad_along_width - pad_left
elif padding is 'valid':
    pad_along_height=0
    pad_along_width=0
    pad_top=0
    pad_bottom=0
    pad_left = 0
    pad_right = 0
    
padshp = (x.shape[0]+pad_along_height,x.shape[1]+pad_along_width,x.shape[2])
pad = [pad_top,pad_bottom,pad_left,pad_right]
kernel = model.layers[1].get_weights()[0]
bias = model.layers[1].get_weights()[1]
x2 = pad2d(x,0,pad).reshape(padshp)

y2 = conv2d(x2,kernel,bias,stride,dilation,np.zeros(y.shape))
print(np.max(np.abs(y.flatten()-y2.flatten())))


1.8536409873703974e-07


In [306]:
layer.get_config()

{'name': 'conv2d_20',
 'trainable': True,
 'filters': 15,
 'kernel_size': (2, 4),
 'strides': (1, 1),
 'padding': 'valid',
 'data_format': 'channels_last',
 'dilation_rate': (2, 2),
 'activation': 'linear',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [376]:
inshp = (31,21,5)
x = np.random.random(inshp)
y = np.max(x,tuple(np.arange(x.ndim-1)))

a = keras.layers.Input(inshp)
b = keras.layers.GlobalMaxPooling2D()(a)
model = keras.models.Model(inputs=a,outputs=b)
y1 = model.predict(x[np.newaxis,...])

def glomax(inputs):
    fin = inputs.flatten()
    chan = inputs.shape[-1]
    outputs = np.zeros(chan)
    for i in range(chan):
        outputs[i] = fin[i]
    for i in range(0,fin.size,chan):
        for j in range(chan):
            if outputs[j]<fin[i+j]:
                outputs[j] = fin[i+j]
    return outputs
            
y2 = glomax(x)
print(np.max(np.abs(y-y1)))
print(np.max(np.abs(y2-y1)))

1.910121461001779e-08
1.910121461001779e-08


In [81]:
shape1 = (None,5,4,2)
shape2 = (None,5,4,1)
output_shape = list(shape1[:-len(shape2)])
for i, j in zip(shape1[-len(shape2):], shape2):
    if i is None or j is None:
        output_shape.append(None)
    elif i == 1:
        output_shape.append(j)
    elif j == 1:
        output_shape.append(i)
    else:
        if i != j:
            raise ValueError('Operands could not be broadcast '
                             'together with shapes ' +
                             str(shape1) + ' ' + str(shape2))
        output_shape.append(i)
set([shape1,shape2]) # check if all inputs are the same shape

{(None, 5, 4, 1), (None, 5, 4, 2)}

In [12]:
### bidirectional

inshape = (8,32)
init1 = keras.initializers.Constant(.04)
init2 = keras.initializers.Identity()
af = keras.layers.Input(inshape)
bf = keras.layers.LSTM(5, return_sequences=True, return_state=False, kernel_initializer=init1, recurrent_initializer=init2)(af)
fwmodel = keras.models.Model(inputs=af, outputs=bf)

ab = keras.layers.Input(inshape)
bb = keras.layers.LSTM(5, return_sequences=True, go_backwards=True, kernel_initializer=init1, recurrent_initializer=init2)(ab)
bwmodel = keras.models.Model(inputs=ab, outputs=bb)

x = np.random.random(inshape)
x= x[np.newaxis,...]
yfw = fwmodel.predict(x)
ybw = bwmodel.predict(x)
xf = np.flip(x,axis=1)
yffw = fwmodel.predict(xf)
yfbw = bwmodel.predict(xf)
print(np.max(np.abs(yfw-ybw)))
print(np.max(np.abs(yfw-yffw)))
print(np.max(np.abs(ybw-yffw)))
print(np.max(np.abs(ybw-yfbw)))



0.065538704
0.065538704
0.0
0.065538704


In [9]:
yfw

array([[[0.2542965 , 0.2542965 , 0.2542965 , 0.2542965 , 0.2542965 ],
        [0.3870733 , 0.3870733 , 0.3870733 , 0.3870733 , 0.3870733 ],
        [0.45849904, 0.45849904, 0.45849904, 0.45849904, 0.45849904],
        [0.55371416, 0.55371416, 0.55371416, 0.55371416, 0.55371416],
        [0.54693186, 0.54693186, 0.54693186, 0.54693186, 0.54693186],
        [0.5663025 , 0.5663025 , 0.5663025 , 0.5663025 , 0.5663025 ],
        [0.6190373 , 0.6190373 , 0.6190373 , 0.6190373 , 0.6190373 ],
        [0.60740703, 0.60740703, 0.60740703, 0.60740703, 0.60740703]]],
      dtype=float32)

In [178]:
inshape = (10,11,12)
axis=2
init = keras.initializers.RandomUniform(minval=.1, maxval=2.0)
center=False
scale=False
a = keras.layers.Input(inshape)
b = keras.layers.BatchNormalization(axis=axis, center=center, scale=scale, beta_initializer=init,
                                    gamma_initializer=init,  moving_variance_initializer=init, moving_mean_initializer=init)(a)
model = keras.models.Model(inputs=a, outputs=b)
epsilon = .001
x = np.random.random(inshape)
x1 = x[np.newaxis,...]
y1 = model.predict(x1)

def batchnorm(inputs, mean,variance, gamma,beta,epsilon,axis):
    fin = inputs.flatten()
    shp = inputs.shape
    offset = 1
    for i in range(axis+1,inputs.ndim,1):
        offset *= shp[i]
    step = shp[axis]
    y = np.zeros(inputs.size)
    stdev = np.sqrt(variance+epsilon)
    for i in range(inputs.size):
        y[i] = (fin[i] - mean[(i//offset)%shp[axis]]) / stdev[(i//offset)%shp[axis]] *\
        gamma[(i//offset)%shp[axis]] + beta[(i//offset)%shp[axis]]
    return y
layer = model.layers[1]
if center and scale:
    gamma = layer.get_weights()[0]
    beta = layer.get_weights()[1]
    mean = layer.get_weights()[2] 
    variance = layer.get_weights()[3]
elif center:
    beta = layer.get_weights()[0]
    mean = layer.get_weights()[1]
    variance = layer.get_weights()[2]
    gamma = np.ones(mean.shape)
elif scale:
    gamma = layer.get_weights()[0]
    mean = layer.get_weights()[1] 
    variance = layer.get_weights()[2]
    beta = np.zeros(mean.shape)
else:
    mean = layer.get_weights()[0] 
    variance = layer.get_weights()[1]
    beta = np.zeros(mean.shape)
    gamma = np.ones(mean.shape)
    
    
y = batchnorm(x,mean,variance,gamma,beta,epsilon,axis-1)
np.max(np.abs(y.flatten()-y1.flatten()))


4.2924625232032554e-07

In [202]:
inshp = (10,20)
input_dim = 20
output_dim = 30
a = keras.layers.Input(inshp)
b = keras.layers.Embedding(input_dim=input_dim,output_dim=output_dim)(a)
model = keras.models.Model(inputs=a,outputs=b)


def embed(inputs,kernel):
    fin = inputs.flatten().astype(int)
    output_dim = kernel.shape[1]
    outputs = np.zeros(inputs.size*output_dim)
    for i in range(inputs.size):
        for j in range(output_dim):
            outputs[i*output_dim + j] = kernel.flatten()[fin[i]*output_dim+j]
    return outputs
layer = model.layers[1]
kernel = layer.get_weights()[0]
x = 10*np.random.random(inshp).astype(int)
x1 = x[np.newaxis,...]
y1 = model.predict(x1)
y = embed(x,kernel)

np.max(np.abs(y.flatten()-y1.flatten()))

0.0

In [192]:
y1.shape

(1, 10, 20, 30)

In [204]:
layer.output_shape

(None, 10, 20, 30)

In [30]:
a = keras.layers.BatchNormalization()
a.get_config()

{'name': 'batch_normalization_1',
 'trainable': True,
 'axis': -1,
 'momentum': 0.99,
 'epsilon': 0.001,
 'center': True,
 'scale': True,
 'beta_initializer': {'class_name': 'Zeros', 'config': {}},
 'gamma_initializer': {'class_name': 'Ones', 'config': {}},
 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}},
 'moving_variance_initializer': {'class_name': 'Ones', 'config': {}},
 'beta_regularizer': None,
 'gamma_regularizer': None,
 'beta_constraint': None,
 'gamma_constraint': None}

In [38]:
a = [1,2,3]
b = 4
bool(b not in a and b is not None)

True

In [25]:
### types, names, io

def layer_type(layer):
    return str(layer.__class__).split('.')[-1][0:-2]

def get_all_io_names(model):
    a = [get_layer_io_names(layer) for layer in model.layers]
    return list(set(flatten(a)))

def get_layer_num_io(layer):
    num_inputs = 0
    error = False
    while not error:
        try:
            layer.get_input_at(num_inputs)
            num_inputs +=1
        except ValueError:
            error = True
    
    num_outputs = 0
    error = False
    while not error:
        try:
            layer.get_output_at(num_outputs)
            num_outputs +=1
        except ValueError:
            error = True
    return num_inputs, num_outputs

def get_layer_io_names(layer):
    num_inputs, num_outputs = get_layer_num_io(layer)
    inputs = []
    # num_inputs>1 -> shared layer
    for i in range(num_inputs):
        # is the input a list?
        if isinstance(layer.get_input_at(i), list):
            temp_list = []
            list_length = len(layer.get_input_at(i))
            for j in range(list_length):
                name = str(layer.get_input_at(i)[j]).split()[0].split('"')[1].split('/')[0].split(':')[0]
                temp_list.append(name)
            inputs.insert(i,temp_list)
        else:
            name = str(layer.get_input_at(i)).split()[0].split('"')[1].split('/')[0].split(':')[0]
            inputs.insert(i,name)
    
    outputs = []       
    for i in range(num_outputs):
        # is the output a list?
        if isinstance(layer.get_output_at(i), list):
            temp_list = []
            list_length = len(layer.get_output_at(i))
            for j in range(list_length):
                name = str(layer.get_output_at(i)[j]).split()[0].split('"')[1].split('/')[0].split(':')[0]
                temp_list.append(name)
            outputs.insert(i,temp_list)
        else:
            name = str(layer.get_output_at(i)).split()[0].split('"')[1].split('/')[0].split(':')[0]
            outputs.insert(i,name)

    return inputs, outputs

def get_model_io_names(model):
    num_inputs = len(model.inputs)
    num_outputs = len(model.outputs)
    inputs = []
    outputs = []
    for i in range(num_inputs):
        nm = str(model.inputs[i]).split()[0].split('"')[1].split('/')[0].split(':')[0]
        inputs.append(nm)
    for i in range(num_outputs):
        nm = str(model.outputs[i]).split()[0].split('"')[1].split('/')[0].split(':')[0]
        outputs.append(nm)
    return inputs, outputs

def flatten(x):
        if isinstance(x, list) or isinstance(x, tuple):
            return [a for i in x for a in flatten(i)]
        else:
            return [x]

In [ ]:
import time
inp1 = np.random.random(inshape)
inp2 = np.random.random(inshape2)
inp = [inp1[np.newaxis,...],inp2[np.newaxis,...]]
num_tests = 10
t1 = time.perf_counter_ns()
for i in range(num_tests):
    outp = model.predict(inp)
t2 = time.perf_counter_ns()
print((t2-t1)/1000/num_tests)